<a href="https://colab.research.google.com/github/sallumandya1995/grammar_coorection/blob/main/_Q%26A_on_PDF_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.youtube.com/watch?v=scJsty_DR3o
# https://www.numpyninja.com/post/question-answering-through-bert-in-10-steps
# https://huggingface.co/clips/mfaq
# https://www.youtube.com/watch?v=OlhNZg4gOvA

# https://haystack.deepset.ai/tutorials/01_basic_qa_pipeline

# https://colab.research.google.com/github/kstathou/vector_engine/blob/master/notebooks/001_vector_search.ipynb#scrollTo=aecWwyrutyef
# https://www.youtube.com/watch?v=w1dMEWm7jBc

In [ ]:
#to watch

# https://www.youtube.com/watch?v=ZxR38An5TQE


# https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb


##from official website

# Notebook [1]: First steps with cdQA

This notebook shows how to use the `cdQA` pipeline to perform question answering on a custom dataset.

***Note:*** *If you are using colab, you will need to install `cdQA` by executing `!pip install cdqa` in a cell.*

In [ ]:
 

!git clone https://github.com/cdqa-suite/cdQA.git
%cd cdQA
!pip install -e .

fatal: destination path 'cdQA' already exists and is not an empty directory.
/content/cdQA
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/cdQA
  Attempting uninstall: cdqa
    Found existing installation: cdqa 1.3.9
    Can't uninstall 'cdqa'. No files were found to uninstall.
  Running setup.py develop for cdqa


In [ ]:
import cdqa
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Download pre-trained reader model and example dataset

In [ ]:
from cdqa.utils.download import download_model, download_bnpp_data

download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='./models')

In [ ]:
!pwd

/content


### Visualize the dataset

In [ ]:
df = pd.read_csv('./data/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df.head()

# df=df1
# df.head()

,date,title,category,link,abstract,paragraphs
0,13.05.2019,The banking jobs : Assistant Vice President – ...,Careers,https://group.bnpparibas/en/news/banking-jobs-...,Within the Group’s Corporate and Institutional...,[I manage a team in charge of designing and im...
1,13.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[With François Hollande, Chairman of French fo..."
2,13.05.2019,"""The bank with an IT budget of more than EUR6 ...",Group,https://group.bnpparibas/en/news/the-bank-budg...,"Interview with Jean-Laurent Bonnafé, Director ...","[We did the groundwork between 2012 and 2016, ..."
3,10.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[As part of the ‘United Tech of Europe’ theme,..."
4,10.05.2019,When Artificial Intelligence participates in r...,Careers,https://group.bnpparibas/en/news/artificial-in...,As the competition to attract talent intensifi...,[Online recruitment is already the norm. Accor...


In [ ]:
# df.columns

# Index(['date', 'title', 'category', 'link', 'abstract', 'paragraphs'], dtype='object')

df = df.drop(df.columns.difference(['title', 'paragraphs']), axis=1)
df.head()

,title,paragraphs
0,The banking jobs : Assistant Vice President – ...,[I manage a team in charge of designing and im...
1,BNP Paribas at #VivaTech : discover the progra...,"[With François Hollande, Chairman of French fo..."
2,"""The bank with an IT budget of more than EUR6 ...","[We did the groundwork between 2012 and 2016, ..."
3,BNP Paribas at #VivaTech : discover the progra...,"[As part of the ‘United Tech of Europe’ theme,..."
4,When Artificial Intelligence participates in r...,[Online recruitment is already the norm. Accor...


In [ ]:
# df.head()
# df['paragraphs'] = df['title'] + df['Key-Words']


# df.drop('Key-Words', axis=1, inplace=True)
# df.drop('title', axis=1, inplace=True)

 

,field_solutiondescriptionimage,paragraphs
0,/sites/default/files/img_35.jpg,"$finance{'sector': 'financial servicesdata', '..."
1,/sites/default/files/img_74.jpg,House_price_prediction_final['house_price_pred...
2,/sites/default/files/img_80.jpg,Diabetes_Patients{'sector': 'lifescience healt...
3,/sites/default/files/img_85.jpg,"21nov{'sector': 'business servicesdata', 'cate..."
4,/sites/default/files/img_58.jpg,"21nov{'sector': 'business servicesdata', 'cate..."


In [ ]:
# data.to_csv("DataWithKeywords.csv")

### Instantiate the cdQA pipeline from a pre-trained reader model

In [ ]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

### Execute a query

In [ ]:
query = 'Since when does the Excellence Program of BNP Paribas exist?'
prediction = cdqa_pipeline.predict(query)

In [ ]:
prediction

('January 2016',
 'BNP Paribas’ commitment to universities and schools',
 'Since January 2016, BNP Paribas has offered an Excellence Program targeting new Master’s level graduates (BAC+5) who show high potential. The aid program lasts 18 months and comprises three assignments of six months each. It serves as a strong career accelerator that enables participants to access high-level management positions at a faster rate. The program allows participants to discover the BNP Paribas Group and its various entities in France and abroad, build an internal and external network by working on different assignments and receive personalized assistance from a mentor and coaching firm at every step along the way.',
 17.718711955320877)

### Explore predictions

In [ ]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: Since when does the Excellence Program of BNP Paribas exist?
answer: January 2016
title: BNP Paribas’ commitment to universities and schools
paragraph: Since January 2016, BNP Paribas has offered an Excellence Program targeting new Master’s level graduates (BAC+5) who show high potential. The aid program lasts 18 months and comprises three assignments of six months each. It serves as a strong career accelerator that enables participants to access high-level management positions at a faster rate. The program allows participants to discover the BNP Paribas Group and its various entities in France and abroad, build an internal and external network by working on different assignments and receive personalized assistance from a mentor and coaching firm at every step along the way.


# Notebook [2]: Using the PDF converter



This notebook shows how to use the PDF converter to create an input dataframe for the cdQA pipeline from a directory of PDF files.


***Note:*** *To run this notebook you will need to have access to GPU. If you are using colab, you will need to install `cdQA` by executing `!pip install cdqa` in a cell.* 

In [ ]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

### Download pre-trained reader model and PDF files

In [ ]:
# Download model
download_model(model='bert-squad_1.1', dir='./models')


bert_qa.joblib already downloaded


In [ ]:
# Download pdf files from BNP Paribas public news
def download_pdf():
    import os
    import wget
    directory = './data/pdf/'
    models_url = [
      'https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf',
      'https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf',
      'https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf'
    ]

    print('\nDownloading PDF files...')

    if not os.path.exists(directory):
        os.makedirs(directory)
    for url in models_url:
        wget.download(url=url, out=directory)

download_pdf()

### Convert the PDF files into a DataFrame for cdQA pipeline

In [ ]:
df = pdf_converter(directory_path='./data/pdf/')
df.head()

### Instantiate the cdQA pipeline from a pre-trained reader model

In [ ]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

# Fit Retriever to documents
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(bert_model='bert-base-uncased', do_lower_case=True,
                         fp16=False, gradient_accumulation_steps=1,
                         learning_rate=3e-05, local_rank=-1, loss_scale=0,
                         max_answer_length=30, n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=2,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_port='', train_batch_size=12,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1))

 ### Execute a query

In [ ]:
query = 'How many contracts did BNP Paribas Cardif sell in 2019?'
prediction = cdqa_pipeline.predict(query)

3it [00:00, 170.06it/s]
The pre-trained model you are loading is an uncased model but you have set `do_lower_case` to False. We are setting `do_lower_case=True` for you but you may want to check this behavior.


### Explore predictions

In [ ]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: How many contracts did BNP Paribas Cardif sell in 2019?
answer: 140,000
title: 1q19-pr-12648.pdf
paragraph: Insurance recorded a good level of activity with in particular the good performance of the international Savings and Protection Insurance businesses and the good development of the new property and casualty insurance offering in the FRB network via Cardif IARD4 (close to 140,000 contracts sold at the end of March 2019). The business committed to energy transition with a target of 3.5 billion euros in green investments by the end of 2020.  


# Notebook [3]: Training the reader on the SQuAD v1.1 dataset

This notebook shows how to fine-tune a pre-trained BERT model on the SQuAD.

***Note:*** *To run this notebook you will need to have access to GPU. The fine-tuning of the Reader was done with an AWS EC2 p3.2xlarge machine (GPU Tesla V100 16GB). It took about 2 hours to complete (2 epochs on SQuAD 1.1 train was enough to achieve SOTA results on SQuAD 1.1 dev).*

In [ ]:
import os
import torch
import joblib
from cdqa.reader import BertProcessor, BertQA
from cdqa.utils.download import download_squad

/home/supercalculateur/source/andre/cdqa-dev/env-cdqa/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
I1120 11:43:47.615704 140657575868224 file_utils.py:39] PyTorch version 1.2.0 available.


### Download SQuAD datasets

In [ ]:
download_squad(dir='./data')

train-v1.1.json already downloaded
dev-v1.1.json already downloaded

train-v2.0.json already downloaded
dev-v2.0.json already downloaded


### Preprocess SQuAD 1.1 examples

In [ ]:
train_processor = BertProcessor(do_lower_case=True, is_training=True)
train_examples, train_features = train_processor.fit_transform(X='./data/SQuAD_1.1/train-v1.1.json')

I1120 11:43:48.194295 140657575868224 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/supercalculateur/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


### Train the model

In [ ]:
reader = BertQA(train_batch_size=12,
                learning_rate=3e-5,
                num_train_epochs=2,
                do_lower_case=True,
                output_dir='models')

reader.fit(X=(train_examples, train_features))

I1120 11:43:53.164162 140657575868224 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/supercalculateur/.cache/torch/transformers/distributed_-1/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I1120 11:43:53.165523 140657575868224 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": fal

BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased', do_lower_case=True,
       fp16=False, gradient_accumulation_steps=1, learning_rate=3e-05,
       local_rank=-1, loss_scale=0, max_answer_length=30, n_best_size=20,
       no_cuda=False, null_score_diff_threshold=0.0, num_train_epochs=2,
       output_dir='models', predict_batch_size=8, seed=42, server_ip='',
       server_port='', train_batch_size=12, verbose_logging=False,
       version_2_with_negative=False, warmup_proportion=0.1, warmup_steps=0)

### Send model to CPU

In [ ]:
reader.model.to('cpu')
reader.device = torch.device('cpu')

### Save model locally

In [ ]:
joblib.dump(reader, os.path.join(reader.output_dir, 'bert_qa.joblib'))

**haystack**

In [ ]:
%%bash

nvidia-smi

Tue Nov 22 06:59:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

To start, install the latest release of Haystack with `pip`:

In [ ]:
 

!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

## Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack.
Example log message:
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/218_Olenna_Tyrell.txt
Default log level in basicConfig is WARNING so the explicit parameter is not necessary but can be changed easily:

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Document Store

Haystack finds answers to queries within the documents stored in a `DocumentStore`. The current implementations of `DocumentStore` include `ElasticsearchDocumentStore`, `FAISSDocumentStore`,  `SQLDocumentStore`, and `InMemoryDocumentStore`.

**Here:** We recommended Elasticsearch as it comes preloaded with features like [full-text queries](https://www.elastic.co/guide/en/elasticsearch/reference/current/full-text-queries.html), [BM25 retrieval](https://www.elastic.co/elasticon/conf/2016/sf/improved-text-scoring-with-bm25), and [vector storage for text embeddings](https://www.elastic.co/guide/en/elasticsearch/reference/7.6/dense-vector.html).

**Alternatives:** If you are unable to setup an Elasticsearch instance, then follow the [Tutorial 3](https://github.com/deepset-ai/haystack-tutorials/blob/main/tutorials/03_Basic_QA_Pipeline_without_Elasticsearch.ipynb) for using SQL/InMemory document stores.

**Hint**: This tutorial creates a new document store instance with Wikipedia articles on Game of Thrones. However, you can configure Haystack to work with your existing document stores.

### Start an Elasticsearch server locally
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (e.g. in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [ ]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

### Start an Elasticsearch server in Colab

If Docker is not readily available in your environment (e.g. in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

### Create the Document Store

The `ElasticsearchDocumentStore` class will try to open a connection in the constructor, here we wait 30 seconds only to be sure Elasticsearch is ready before continuing:

In [ ]:
import time
time.sleep(30)

Finally, we create the Document Store instance:

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


## Preprocessing of documents

Haystack provides a customizable pipeline for:
 - converting files into texts
 - cleaning texts
 - splitting texts
 - writing them to a Document Store

In this tutorial, we download Wikipedia articles about Game of Thrones, apply a basic cleaning function, and index them in Elasticsearch.

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


# Let's first fetch some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/tutorial1"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is:
# {
#    'content': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
# }
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Pipeline)

# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip to 'data/tutorial1'
INFO:haystack.utils.preprocessing:Converting data/tutorial1/207_Jon_Snow__character_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/358_A_Game_of_Thrones__Genesis.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/334_Rickon_Stark.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/495_Hodor.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/9_Game_of_Thrones_Tapestry.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/10_Beyond_the_Wall__Game_of_Thrones_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/51_Iron_Throne__A_Song_of_Ice_and_Fire_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/195_World_of_A_Song_of_Ice_and_Fire.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/513_Oathbreaker__Game_of_Thrones_.

[<Document: {'content': '\'\'\'Jon Snow\'\'\' is a fictional character in the \'\'A Song of Ice and Fire\'\' series of fantasy novels by American author George R. R. Martin, and its television adaptation \'\'Game of Thrones\'\', in which he is portrayed by English actor Kit Harington. In the novels, he is a prominent point of view character. He is one of the most popular characters in the series, and \'\'The New York Times\'\' cites him as one of the author\'s finest creations. Jon is a main character in the TV series, and his storyline in the season 5 finale generated a strong reaction among viewers. Speculation about the character\'s parentage has also been a popular topic of discussion among fans of both the books and the TV series.\nJon is introduced in 1996\'s \'\'A Game of Thrones\'\' as the illegitimate son of Ned Stark, the honorable lord of Winterfell, an ancient fortress in the North of the fictional continent of Westeros. Knowing his prospects are limited by his status, Jon 

## Initialize Retriever, Reader & Pipeline

### Retriever

Retrievers help narrowing down the scope for the Reader to smaller units of text where a given question could be answered.
They use some simple but fast algorithm.

**Here:** We use Elasticsearch's default BM25 algorithm

**Alternatives:**

- Customize the `BM25Retriever`with custom queries (e.g. boosting) and filters
- Use `TfidfRetriever` in combination with a SQL or InMemory Document store for simple prototyping and debugging
- Use `EmbeddingRetriever` to find candidate documents based on the similarity of embeddings (e.g. created via Sentence-BERT)
- Use `DensePassageRetriever` to use different embedding models for passage and query (see Tutorial 6)

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
# Alternative: An in-memory TfidfRetriever based on Pandas dataframes for building quick-prototypes with SQLite document store.

# from haystack.nodes import TfidfRetriever
# retriever = TfidfRetriever(document_store=document_store)

### Reader

A Reader scans the texts returned by retrievers in detail and extracts the k best answers. They are based
on powerful, but slower deep learning models.

Haystack currently supports Readers based on the frameworks FARM and Transformers.
With both you can either load a local model or one from Hugging Face's model hub (https://huggingface.co/models).

**Here:** a medium sized RoBERTa QA model using a Reader based on FARM (https://huggingface.co/deepset/roberta-base-squad2)

**Alternatives (Reader):** TransformersReader (leveraging the `pipeline` of the Transformers package)

**Alternatives (Models):** e.g. "distilbert-base-uncased-distilled-squad" (fast) or "deepset/bert-large-uncased-whole-word-masking-squad2" (good accuracy)

**Hint:** You can adjust the model to return "no answer possible" with the no_ans_boost. Higher values mean the model prefers "no answer possible"

#### FARMReader

In [ ]:
from haystack.nodes import FARMReader

# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


#### TransformersReader

Alternative:

In [ ]:
from haystack.nodes import TransformersReader
# reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

### Pipeline

With a Haystack `Pipeline` you can stick together your building blocks to a search pipeline.
Under the hood, `Pipelines` are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases.
To speed things up, Haystack also comes with a few predefined Pipelines. One of them is the `ExtractiveQAPipeline` that combines a retriever and a reader to answer our questions.
You can learn more about `Pipelines` in the [docs](https://haystack.deepset.ai/docs/latest/pipelines).

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

## Voilà! Ask a question!

In [ ]:
# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.88s/ Batches]


In [ ]:
# prediction = pipe.run(query="Who created the Dothraki vocabulary?", params={"Reader": {"top_k": 5}})
# prediction = pipe.run(query="Who is the sister of Sansa?", params={"Reader": {"top_k": 5}})

Now you can either print the object directly:

In [ ]:
from pprint import pprint

pprint(prediction)

# Sample output:
# {
#     'answers': [ <Answer: answer='Eddard', type='extractive', score=0.9919578731060028, offsets_in_document=[{'start': 608, 'end': 615}], offsets_in_context=[{'start': 72, 'end': 79}], document_id='cc75f739897ecbf8c14657b13dda890e', meta={'name': '454_Music_of_Game_of_Thrones.txt'}}, context='...' >,
#                  <Answer: answer='Ned', type='extractive', score=0.9767240881919861, offsets_in_document=[{'start': 3687, 'end': 3801}], offsets_in_context=[{'start': 18, 'end': 132}], document_id='9acf17ec9083c4022f69eb4a37187080', meta={'name': '454_Music_of_Game_of_Thrones.txt'}}, context='...' >,
#                  ...
#                ]
#     'documents': [ <Document: content_type='text', score=0.8034909798951382, meta={'name': '332_Sansa_Stark.txt'}, embedding=None, id=d1f36ec7170e4c46cde65787fe125dfe', content='\n===\'\'A Game of Thrones\'\'===\nSansa Stark begins the novel by being betrothed to Crown ...'>,
#                    <Document: content_type='text', score=0.8002150354529785, meta={'name': '191_Gendry.txt'}, embedding=None, id='dd4e070a22896afa81748d6510006d2', 'content='\n===Season 2===\nGendry travels North with Yoren and other Night's Watch recruits, including Arya ...'>,
#                    ...
#                  ],
#     'no_ans_gap':  11.688868522644043,
#     'node_id': 'Reader',
#     'params': {'Reader': {'top_k': 5}, 'Retriever': {'top_k': 5}},
#     'query': 'Who is the father of Arya Stark?',
#     'root_node': 'Query'
# }

{'answers': [<Answer {'answer': 'Eddard', 'type': 'extractive', 'score': 0.9919580221176147, 'context': "s Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leaves,", 'offsets_in_document': [{'start': 147, 'end': 153}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_id': 'ba2a8e87ddd95e380bec55983ee7d55f', 'meta': {'name': '43_Arya_Stark.txt'}}>,
             <Answer {'answer': 'Ned', 'type': 'extractive', 'score': 0.97672438621521, 'context': "\n====Season 1====\nArya accompanies her father Ned and her sister Sansa to King's Landing. Before their departure, Arya's half-brother Jon Snow gifts A", 'offsets_in_document': [{'start': 46, 'end': 49}], 'offsets_in_context': [{'start': 46, 'end': 49}], 'document_id': '180c2a6b36369712b361a80842e79356', 'meta': {'name': '43_Arya_Stark.txt'}}>,
             <Answer {'answer': 'Lord Eddard Stark', 'type': 'extractive', 'score': 0.8930398821830

Or use a util to simplify the output:

In [ ]:
from haystack.utils import print_answers

# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(prediction, details="minimum")


Query: Who is the father of Arya Stark?
Answers:
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's "
                   'half-brother Jon Snow gifts A'},
    {   'answer': 'Lord Eddard Stark',
        'context': 'ark daughters.\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.'},
    {   'answer': 'Joffrey',
        'context': 'laying with one of his wooden toys.\n'
                   "After Eddard discovers the 

In [ ]:
# https://github.com/deepset-ai/haystack-tutorials

In [ ]:
# https://github.com/deepset-ai/haystack-tutorials/blob/main/tutorials/03_Basic_QA_Pipeline_without_Elasticsearch.ipynb

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
!pip install --upgrade "urllib3==1.25.4"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.12
    Uninstalling urllib3-1.26.12:
      Successfully uninstalled urllib3-1.26.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
docker 6.0.1 requires urllib3>=1.26.0, but you have urllib3 1.25.4 which is incompatible.


In [ ]:
from haystack.utils import fetch_archive_from_http
# from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/tutorial3"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is: {"name": "", "content": ""}

# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the docs to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip to 'data/tutorial3'
INFO:haystack.utils.preprocessing:Converting data/tutorial3/207_Jon_Snow__character_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/358_A_Game_of_Thrones__Genesis.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/334_Rickon_Stark.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/495_Hodor.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/9_Game_of_Thrones_Tapestry.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/10_Beyond_the_Wall__Game_of_Thrones_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/51_Iron_Throne__A_Song_of_Ice_and_Fire_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/195_World_of_A_Song_of_Ice_and_Fire.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/513_Oathbreaker__Game_of_Thrones_.

[<Document: {'content': '\'\'\'Jon Snow\'\'\' is a fictional character in the \'\'A Song of Ice and Fire\'\' series of fantasy novels by American author George R. R. Martin, and its television adaptation \'\'Game of Thrones\'\', in which he is portrayed by English actor Kit Harington. In the novels, he is a prominent point of view character. He is one of the most popular characters in the series, and \'\'The New York Times\'\' cites him as one of the author\'s finest creations. Jon is a main character in the TV series, and his storyline in the season 5 finale generated a strong reaction among viewers. Speculation about the character\'s parentage has also been a popular topic of discussion among fans of both the books and the TV series.\nJon is introduced in 1996\'s \'\'A Game of Thrones\'\' as the illegitimate son of Ned Stark, the honorable lord of Winterfell, an ancient fortress in the North of the fictional continent of Westeros. Knowing his prospects are limited by his status, Jon 

INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'c8b51f62e0fccac8361c4464cc2c8f70' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '9e9a3181b6bc168b4a25429b641e8c86' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '5d79fbf801011475553a09ab068f02e2' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '7ed12f389f7f085bb30c7d00abd26f81' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '9e9a3181b6bc168b4a25429b641e8c86' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '9e9a3181b6bc168b4a25429b641e8c86' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '5d79fbf801011475553a09ab068f02e2' already exists in index 'document'

In [ ]:
# An in-memory TfidfRetriever based on Pandas dataframes
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

INFO:haystack.nodes.retriever.sparse:Found 2357 candidate paragraphs from 2357 docs in DB


In [ ]:
from haystack.nodes import FARMReader


# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# Now you can either print the object directly...
from pprint import pprint

pprint(prediction)

# Sample output:
# {
#     'answers': [ ,
#                  ,
#                  ...
#                ]
#     'documents': [ ,
#                    ,
#                    ...
#                  ],
#     'no_ans_gap':  11.688868522644043,
#     'node_id': 'Reader',
#     'params': {'Reader': {'top_k': 5}, 'Retriever': {'top_k': 5}},
#     'query': 'Who is the father of Arya Stark?',
#     'root_node': 'Query'
# }

{'answers': [<Answer {'answer': 'Eddard', 'type': 'extractive', 'score': 0.9919580221176147, 'context': "s Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leaves,", 'offsets_in_document': [{'start': 147, 'end': 153}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_id': 'ba2a8e87ddd95e380bec55983ee7d55f', 'meta': {'name': '43_Arya_Stark.txt'}}>,
             <Answer {'answer': 'Ned', 'type': 'extractive', 'score': 0.97672438621521, 'context': "\n====Season 1====\nArya accompanies her father Ned and her sister Sansa to King's Landing. Before their departure, Arya's half-brother Jon Snow gifts A", 'offsets_in_document': [{'start': 46, 'end': 49}], 'offsets_in_context': [{'start': 46, 'end': 49}], 'document_id': '180c2a6b36369712b361a80842e79356', 'meta': {'name': '43_Arya_Stark.txt'}}>,
             <Answer {'answer': 'Lord Eddard Stark', 'type': 'extractive', 'score': 0.8930398821830

In [ ]:
# ...or use a util to simplify the output
from haystack.utils import print_answers


# Change `minimum` to `medium` or `all` to control the level of detail
print_answers(prediction, details="minimum")


Query: Who is the father of Arya Stark?
Answers:
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's "
                   'half-brother Jon Snow gifts A'},
    {   'answer': 'Lord Eddard Stark',
        'context': 'ark daughters.\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.'},
    {   'answer': 'Joffrey',
        'context': 'laying with one of his wooden toys.\n'
                   "After Eddard discovers the 

In [ ]:
import pandas as pd

data = pd.read_excel('/content/cms_solutions_nov21.xlsx')

# /content/cleansheet (1).xlsx

In [ ]:

data["clean_description"] = data["field_solutiondescription"].str.replace("<strong>", " ")

data["clean_description"] = data["field_solutiondescription"].str.replace("</strong>", " ")

data["clean_description"] = data["field_solutiondescription"].str.replace("</p>", " ")

data["clean_description"] = data["field_solutiondescription"].str.replace("<p>", " ")

In [ ]:

def clean_description(text):



  text = text.str.replace("<strong>", " ")

  text = text.str.replace("</strong>", " ")

  text = text.str.replace("</p>", " ")

  text = text.str.replace("<p>", " ")
  
  return text

data["clean_description"] = clean_description(data["field_solutiondescription"])

In [ ]:
data["clean_description"]

0         Sector:  Financial Services   Data Category:...
1                            House_price_prediction_final
2         Sector:  Lifescience, HealthCare   Data Cate...
3         Sector:  Business Services   Data Category: ...
4         Sector:  Business Services   Data Category: ...
                              ...                        
3387    <p style="margin-bottom:11px"><span style="fon...
3388    <p style="margin-bottom:11px"><span style="fon...
3389     Enables ability to determine most productive ...
3390     <span><span><span> <span lang="EN-GB" xml:lan...
3391     &lt;p style=&quot;margin-bottom:7px; text-ali...
Name: clean_description, Length: 3392, dtype: object

In [ ]:
data.columns

Index(['record', 'title', 'key_words', 'field_solutiondescription',
       'field_solutiondescriptionimage', 'field_solutiondescriptionvideo',
       'field_solutioninsightexchange', 'field_solutionlink',
       'field_solutioninfodescription', 'field_solutioninfomedia', 'nid',
       'field_solutiondescriptionimage_1', 'field_solutiondescriptionvideo_1',
       'field_solutioninsightexchange_1', 'field_solutioninfomedia_1',
       'field_solutioninfodescription2', 'field_solutionmoreinforefimages',
       'field_solutiontags', 'field_solutiondomain', 'field_solutionindustry',
       'field_solutionoutcomeexchange', 'field_solutionshortdescription',
       'field_solutionaccesslink', 'field_solutiontype',
       'field_solutiondataurl', 'clean_description'],
      dtype='object')

In [ ]:
data.head()

,record,title,key_words,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,field_solutioninfomedia,...,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl,clean_description
0,NaN,$finance,NaN,<p><strong>Sector:</strong> Financial Services...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN,Sector: Financial Services Data Category:...
1,NaN,House_price_prediction_final,NaN,House_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN,House_price_prediction_final
2,NaN,Diabetes_Patients,NaN,"<p><strong>Sector:</strong> Lifescience, Healt...",/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN,"Sector: Lifescience, HealthCare Data Cate..."
3,NaN,21nov,NaN,<p><strong>Sector:</strong> Business Services<...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN,Sector: Business Services Data Category: ...
4,NaN,21nov,NaN,<p><strong>Sector:</strong> Business Services<...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN,Sector: Business Services Data Category: ...


In [ ]:
data.to_csv("123.csv")

In [ ]:
data.head()

,record,title,key_words,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,field_solutioninfomedia,...,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl,clean_description
0,NaN,$finance,NaN,<p><strong>Sector:</strong> Financial Services...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN,Sector: Financial Services Data Category:...
1,NaN,House_price_prediction_final,NaN,House_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN,House_price_prediction_final
2,NaN,Diabetes_Patients,NaN,"<p><strong>Sector:</strong> Lifescience, Healt...",/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN,"Sector: Lifescience, HealthCare Data Cate..."
3,NaN,21nov,NaN,<p><strong>Sector:</strong> Business Services<...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN,Sector: Business Services Data Category: ...
4,NaN,21nov,NaN,<p><strong>Sector:</strong> Business Services<...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,NaN,...,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN,Sector: Business Services Data Category: ...


In [ ]:
def sector_category(column):

    
    df['Sector'] = df['clean_description'].str.extract('Sector: (.*?)Data', expand=False)
    df['Category'] = df['clean_description'].str.extract('Category: (.*?)Data', expand=False)
    df['Characteristic'] = df['clean_description'].str.extract('Characteristic: (.*?)Potential', expand=False)
    df['Potential Cases'] = df['clean_description'].str.extract('Cases: (.*?)$', expand=False)
    
    return df

In [ ]:
data = sector_category(data)

In [ ]:
d1=data['clean_description']

In [ ]:
d1.to_csv("13.csv")

In [ ]:
import pandas as pd

data = pd.read_excel('/content/cleansheet (1).xlsx')
data.head()
# 

,Unnamed: 0,record,title,key_words,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
0,0,NaN,$finance,NaN,Sector: Financial ServicesData Category: Finan...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN
1,1,NaN,House_price_prediction_final,NaN,House_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN
2,2,NaN,Diabetes_Patients,NaN,"Sector: Lifescience, HealthCareData Category: ...",/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN
3,3,NaN,21nov,NaN,Sector: Business ServicesData Category: Adapti...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN
4,4,NaN,21nov,NaN,Sector: Business ServicesData Category: Adapti...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN


In [ ]:
data.columns

Index(['Unnamed: 0', 'record', 'title', 'key_words',
       'field_solutiondescription', 'field_solutiondescriptionimage',
       'field_solutiondescriptionvideo', 'field_solutioninsightexchange',
       'field_solutionlink', 'field_solutioninfodescription',
       'field_solutioninfomedia', 'nid', 'field_solutiondescriptionimage_1',
       'field_solutiondescriptionvideo_1', 'field_solutioninsightexchange_1',
       'field_solutioninfomedia_1', 'field_solutioninfodescription2',
       'field_solutionmoreinforefimages', 'field_solutiontags',
       'field_solutiondomain', 'field_solutionindustry',
       'field_solutionoutcomeexchange', 'field_solutionshortdescription',
       'field_solutionaccesslink', 'field_solutiontype',
       'field_solutiondataurl'],
      dtype='object')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Unnamed: 0,record,title,key_words,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
0,0,NaN,$finance,NaN,sector financial servicesdata category financi...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN
1,1,NaN,House_price_prediction_final,NaN,house_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN
2,2,NaN,Diabetes_Patients,NaN,sector lifescience healthcaredata category hea...,/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN
3,3,NaN,21nov,NaN,sector business servicesdata category adaption...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN
4,4,NaN,21nov,NaN,sector business servicesdata category adaption...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN


In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_excel("cms_solutions_nov21.xlsx")
df.head()

,record,title,key_words,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,field_solutioninfomedia,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
0,NaN,$finance,NaN,<p><strong>Sector:</strong> Financial Services...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN
1,NaN,House_price_prediction_final,NaN,House_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN
2,NaN,Diabetes_Patients,NaN,"<p><strong>Sector:</strong> Lifescience, Healt...",/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN
3,NaN,21nov,NaN,<p><strong>Sector:</strong> Business Services<...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN
4,NaN,21nov,NaN,<p><strong>Sector:</strong> Business Services<...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN


In [ ]:
df.iloc[[3391]]

,record,title,key_words,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,field_solutioninfomedia,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
3391,NaN,Supplier Operational Performance,NaN,<p>&lt;p style=&quot;margin-bottom:7px; text-a...,/sites/default/files/2020-04/img_59.jpg,SupplierOperationPerformance.mp4,97.0,http://ndaps10/#/signin?externalRedirect=%2Fvi...,<p>&lt;p&gt;&amp;nbsp;&lt;/p&gt;</p>\n<p>&lt;p...,<p> </p>\n\n<p> </p>,...,<p> </p>\n\n<p> </p>\n,/sites/default/files/2019-07/supplier-operatio...,"supplier efficiency, operational risk, fill ra...",Procurement - Manufacturing,Manufacturing,Supplier Management - Procurement,tagline,NaN,Insight,NaN


In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    str1 = str(text)
    clean = re.compile('<.*?>')
    return re.sub(clean, '', str1)

In [ ]:
for i in range(len(df)):
    a = df["field_solutiondescription"][i]
    b = remove_html_tags(a)
    df['field_solutiondescription'][i] = b

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df['field_solutiondescription'][3391]

'&lt;p style=&quot;margin-bottom:7px; text-align:justify&quot;&gt;&lt;span style=&quot;font-size:11pt&quot;&gt;&lt;span style=&quot;line-height:110%&quot;&gt;&lt;span style=&quot;font-family:Calibri,sans-serif&quot;&gt;&lt;b&gt;&lt;span lang=&quot;EN-US&quot; style=&quot;font-size:9.0pt&quot;&gt;&lt;span style=&quot;line-height:110%&quot;&gt;&lt;span style=&quot;font-family:&amp;quot;Verdana&amp;quot;,&amp;quot;sans-serif&amp;quot;&quot;&gt;Solution Provider&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/b&gt;&lt;span lang=&quot;EN-US&quot; style=&quot;font-size:9.0pt&quot;&gt;&lt;span style=&quot;line-height:110%&quot;&gt;&lt;span style=&quot;font-family:&amp;quot;Verdana&amp;quot;,&amp;quot;sans-serif&amp;quot;&quot;&gt;: Capgemini&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/p&gt;\n&lt;p style=&quot;margin-bottom:7px; text-align:justify&quot;&gt;&lt;span style=&quot;font-size:11pt&quot;&gt;&lt;span style=&quot;line-height:110%&quot;&gt;&lt;span style

In [ ]:
df.to_csv("cleanedData.csv")

In [ ]:
df["field_solutiondescription"]

0       Sector: Financial ServicesData Category: Finan...
1                            House_price_prediction_final
2       Sector: Lifescience, HealthCareData Category: ...
3       Sector: Business ServicesData Category: Adapti...
4       Sector: Business ServicesData Category: Adapti...
                              ...                        
3387    Solution Provider: Capgemini\n\nSelecting the ...
3388    Solution Provider: Capgemini\n\nMost organizat...
3389    Enables ability to determine most productive a...
3390    Solution Provider: Capgemini\n\nIn the consume...
3391    &lt;p style=&quot;margin-bottom:7px; text-alig...
Name: field_solutiondescription, Length: 3392, dtype: object

In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv("cleanedData.csv")
df.tail()

,Unnamed: 0,record,title,key_words,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
3387,3387,NaN,Supplier Portfolio Restructuring,NaN,Solution Provider: Capgemini\n\nSelecting the ...,/sites/default/files/2020-04/img_61.jpg,SupplierPortfolioRestructuring.mp4,102.0,http://ndaps10/views/SupplierReduction/Supplie...,<p> </p>\n\n<p> </p>,...,<p> </p>\n\n<p> </p>\n,/sites/default/files/2019-07/supplier-portfoli...,"distributor, provider, dealer, merchandiser, m...",Procurement - Manufacturing,Manufacturing,Supplier Management - Procurement,Improve procurement efficiency,NaN,Insight,NaN
3388,3388,NaN,Supplier Financial Risk,NaN,Solution Provider: Capgemini\n\nMost organizat...,/sites/default/files/2020-04/img_57.jpg,SupplierFinancialRisk.mp4,98.0,http://ndaps10/#/signin?externalRedirect=%2Fvi...,<p> </p>\n\n<p> </p>,...,<p> </p>\n\n<p> </p>\n,/sites/default/files/2019-07/supplier-financia...,"supplier quality, delivery, cost reduction, s...",Procurement - Manufacturing,Manufacturing,Supplier Management - Procurement,Assess suppliers’ financial stability,NaN,Insight,NaN
3389,3389,NaN,Assortment Optimization,NaN,Enables ability to determine most productive a...,/sites/default/files/2020-04/img_72.jpg,NaN,104.0,https://idv.dataventures.com/iDV/signon.aspx,<p><strong>Who to contact</strong>: 890support...,...,NaN,/sites/default/files/2019-12/AO.png,"product mix, strategy, segregation, allocation...",Commerce - Retail,Retail,Category Effectiveness - Commerce,Maximize sales with optimized assortment recom...,NaN,Tool,NaN
3390,3390,NaN,Brand Equity,NaN,Solution Provider: Capgemini\n\nIn the consume...,/sites/default/files/2020-04/img_9_1.jpg,Brand+Equity.mp4,19.0,https://app.powerbi.com/view?r=eyJrIjoiOWNhYTA...,<p> </p>\n\n<p> </p>,...,<p> </p>\n\n<p> </p>,"/sites/default/files/2019-12/BE1.png, /sites/d...","brand image, brand strength, brand loyalty, br...",Marketing-ConsumerProducts,ConsumerProduct,Consumer Engagement-Marketing,Near real-time brand equity comparison with co...,NaN,Insight,NaN
3391,3391,NaN,Supplier Operational Performance,NaN,&lt;p style=&quot;margin-bottom:7px; text-alig...,/sites/default/files/2020-04/img_59.jpg,SupplierOperationPerformance.mp4,97.0,http://ndaps10/#/signin?externalRedirect=%2Fvi...,<p>&lt;p&gt;&amp;nbsp;&lt;/p&gt;</p>\n<p>&lt;p...,...,<p> </p>\n\n<p> </p>\n,/sites/default/files/2019-07/supplier-operatio...,"supplier efficiency, operational risk, fill ra...",Procurement - Manufacturing,Manufacturing,Supplier Management - Procurement,tagline,NaN,Insight,NaN


In [ ]:
len(df['field_solutiondescription'])

3392

In [ ]:
df.head()

,Unnamed: 0,record,title,key_words,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
0,0,NaN,$finance,NaN,Sector: Financial ServicesData Category: Finan...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN
1,1,NaN,House_price_prediction_final,NaN,House_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN
2,2,NaN,Diabetes_Patients,NaN,"Sector: Lifescience, HealthCareData Category: ...",/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN
3,3,NaN,21nov,NaN,Sector: Business ServicesData Category: Adapti...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN
4,4,NaN,21nov,NaN,Sector: Business ServicesData Category: Adapti...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

df['field_solutiondescription'] = df['field_solutiondescription'].astype(str)
df['field_solutiondescription']=df['field_solutiondescription'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['field_solutiondescription']=df['field_solutiondescription'].str.replace('[^\w\s]',' ')
stop_words=set(stopwords.words('english'))

df['field_solutiondescription']=df['field_solutiondescription'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Unnamed: 0,record,title,key_words,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
0,0,NaN,$finance,NaN,sector financial servicesdata category financi...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN
1,1,NaN,House_price_prediction_final,NaN,house_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN
2,2,NaN,Diabetes_Patients,NaN,sector lifescience healthcaredata category hea...,/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN
3,3,NaN,21nov,NaN,sector business servicesdata category adaption...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN
4,4,NaN,21nov,NaN,sector business servicesdata category adaption...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN


In [ ]:
def string_list_dict(dataframe, column_name):
    df2 = pd.DataFrame(columns=['Key-Words'])
    for i in range(len(df)):
        dict1 = {}
        list_1 = df[column_name][i].split()
        if 'sector' in list_1:
            dict1.update({list_1[list_1.index('sector')]: (" ").join(list_1[list_1.index('sector')+1:list_1.index('sector')+3])})
        if 'category' in list_1:
            dict1.update({list_1[list_1.index('category')]: (" ").join(list_1[list_1.index('category')+1:list_1.index('category')+3])})
        if 'characteristic' in list_1:
            dict1.update({list_1[list_1.index('characteristic')]: (" ").join(list_1[list_1.index('characteristic')+1:list_1.index('characteristic')+2])})
        if 'potential' in list_1:
            dict1.update({(" ").join(list_1[list_1.index('potential'):list_1.index('potential')+2]): list_1[list_1.index('potential')+2]})
            df2 = df2.append({'Key-Words':dict1}, ignore_index=True)
        else:
            df2 = df2.append({'Key-Words':list_1}, ignore_index=True)
    return df2

In [ ]:
data = string_list_dict(df, 'field_solutiondescription')
df['Key-Words'] = data['Key-Words']
df.drop(columns=['Unnamed: 0', 'record'], axis = 1, inplace = True)
df.head()

AttributeError: ignored

In [ ]:
df.columns

Index(['Unnamed: 0', 'record', 'title', 'key_words',
       'field_solutiondescription', 'field_solutiondescriptionimage',
       'field_solutiondescriptionvideo', 'field_solutioninsightexchange',
       'field_solutionlink', 'field_solutioninfodescription',
       'field_solutioninfomedia', 'nid', 'field_solutiondescriptionimage_1',
       'field_solutiondescriptionvideo_1', 'field_solutioninsightexchange_1',
       'field_solutioninfomedia_1', 'field_solutioninfodescription2',
       'field_solutionmoreinforefimages', 'field_solutiontags',
       'field_solutiondomain', 'field_solutionindustry',
       'field_solutionoutcomeexchange', 'field_solutionshortdescription',
       'field_solutionaccesslink', 'field_solutiontype',
       'field_solutiondataurl'],
      dtype='object')

In [ ]:
data.to_csv("DataWithKeywords.csv")

NameError: ignored

In [ ]:
df.head()

In [ ]:
#  https://huggingface.co/google/tapas-base-finetuned-wtq
# https://huggingface.co/spaces/katanaml/table-query

In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_excel("cms_solutions_nov21.xlsx")

df.drop(columns=['key_words'], axis = 1, inplace = True)

In [ ]:
df.head()

,record,title,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,field_solutioninfomedia,nid,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
0,NaN,$finance,<p><strong>Sector:</strong> Financial Services...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,NaN,6533,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN
1,NaN,House_price_prediction_final,House_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,NaN,6532,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN
2,NaN,Diabetes_Patients,"<p><strong>Sector:</strong> Lifescience, Healt...",/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,NaN,6531,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN
3,NaN,21nov,<p><strong>Sector:</strong> Business Services<...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,NaN,6530,...,NaN,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN
4,NaN,21nov,<p><strong>Sector:</strong> Business Services<...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,NaN,6529,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN


In [ ]:
df.iloc[[3391]]

,record,title,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,field_solutioninfomedia,nid,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
3391,NaN,Supplier Operational Performance,<p>&lt;p style=&quot;margin-bottom:7px; text-a...,/sites/default/files/2020-04/img_59.jpg,SupplierOperationPerformance.mp4,97.0,http://ndaps10/#/signin?externalRedirect=%2Fvi...,<p>&lt;p&gt;&amp;nbsp;&lt;/p&gt;</p>\n<p>&lt;p...,<p> </p>\n\n<p> </p>,45,...,<p> </p>\n\n<p> </p>\n,/sites/default/files/2019-07/supplier-operatio...,"supplier efficiency, operational risk, fill ra...",Procurement - Manufacturing,Manufacturing,Supplier Management - Procurement,tagline,NaN,Insight,NaN


In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    str1 = str(text)
    clean = re.compile('<.*?>')
    return re.sub(clean, '', str1)

In [ ]:
for i in range(len(df)):
    a = df["field_solutiondescription"][i]
    b = remove_html_tags(a)
    df['field_solutiondescription'][i] = b

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df['field_solutiondescription'][3391]

'&lt;p style=&quot;margin-bottom:7px; text-align:justify&quot;&gt;&lt;span style=&quot;font-size:11pt&quot;&gt;&lt;span style=&quot;line-height:110%&quot;&gt;&lt;span style=&quot;font-family:Calibri,sans-serif&quot;&gt;&lt;b&gt;&lt;span lang=&quot;EN-US&quot; style=&quot;font-size:9.0pt&quot;&gt;&lt;span style=&quot;line-height:110%&quot;&gt;&lt;span style=&quot;font-family:&amp;quot;Verdana&amp;quot;,&amp;quot;sans-serif&amp;quot;&quot;&gt;Solution Provider&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/b&gt;&lt;span lang=&quot;EN-US&quot; style=&quot;font-size:9.0pt&quot;&gt;&lt;span style=&quot;line-height:110%&quot;&gt;&lt;span style=&quot;font-family:&amp;quot;Verdana&amp;quot;,&amp;quot;sans-serif&amp;quot;&quot;&gt;: Capgemini&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/span&gt;&lt;/p&gt;\n&lt;p style=&quot;margin-bottom:7px; text-align:justify&quot;&gt;&lt;span style=&quot;font-size:11pt&quot;&gt;&lt;span style=&quot;line-height:110%&quot;&gt;&lt;span style

In [ ]:
df.to_csv("cleanedData.csv")

In [ ]:
df["field_solutiondescription"]

0       Sector: Financial ServicesData Category: Finan...
1                            House_price_prediction_final
2       Sector: Lifescience, HealthCareData Category: ...
3       Sector: Business ServicesData Category: Adapti...
4       Sector: Business ServicesData Category: Adapti...
                              ...                        
3387    Solution Provider: Capgemini\n\nSelecting the ...
3388    Solution Provider: Capgemini\n\nMost organizat...
3389    Enables ability to determine most productive a...
3390    Solution Provider: Capgemini\n\nIn the consume...
3391    &lt;p style=&quot;margin-bottom:7px; text-alig...
Name: field_solutiondescription, Length: 3392, dtype: object

### Assigning Keywords

In [ ]:
df = pd.read_csv("cleanedData.csv")

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

df['field_solutiondescription'] = df['field_solutiondescription'].astype(str)
df['field_solutiondescription']=df['field_solutiondescription'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['field_solutiondescription']=df['field_solutiondescription'].str.replace('[^\w\s]',' ')
stop_words=set(stopwords.words('english'))

df['field_solutiondescription']=df['field_solutiondescription'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


,Unnamed: 0,record,title,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,field_solutioninfomedia,...,field_solutioninfodescription2,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl
0,0,NaN,$finance,sector financial servicesdata category financi...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN
1,1,NaN,House_price_prediction_final,house_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN
2,2,NaN,Diabetes_Patients,sector lifescience healthcaredata category hea...,/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN
3,3,NaN,21nov,sector business servicesdata category adaption...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN
4,4,NaN,21nov,sector business servicesdata category adaption...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN


In [ ]:
def string_list_dict(dataframe, column_name):
    df2 = pd.DataFrame(columns=['Key-Words'])
    for i in range(len(df)):
        dict1 = {}
        list_1 = df[column_name][i].split()
        if 'sector' in list_1:
            dict1.update({list_1[list_1.index('sector')]: (" ").join(list_1[list_1.index('sector')+1:list_1.index('sector')+3])})
        if 'category' in list_1:
            dict1.update({list_1[list_1.index('category')]: (" ").join(list_1[list_1.index('category')+1:list_1.index('category')+3])})
        if 'characteristic' in list_1:
            dict1.update({list_1[list_1.index('characteristic')]: (" ").join(list_1[list_1.index('characteristic')+1:list_1.index('characteristic')+2])})
        if 'potential' in list_1:
            dict1.update({(" ").join(list_1[list_1.index('potential'):list_1.index('potential')+2]): list_1[list_1.index('potential')+2]})
            df2 = df2.append({'Key-Words':dict1}, ignore_index=True)
        else:
            df2 = df2.append({'Key-Words':list_1}, ignore_index=True)
    return df2

In [ ]:
data = string_list_dict(df, 'field_solutiondescription')
df['Key-Words'] = data['Key-Words']
df.drop(columns=['Unnamed: 0', 'record'], axis = 1, inplace = True)
df.head()

,title,field_solutiondescription,field_solutiondescriptionimage,field_solutiondescriptionvideo,field_solutioninsightexchange,field_solutionlink,field_solutioninfodescription,field_solutioninfomedia,nid,field_solutiondescriptionimage_1,...,field_solutionmoreinforefimages,field_solutiontags,field_solutiondomain,field_solutionindustry,field_solutionoutcomeexchange,field_solutionshortdescription,field_solutionaccesslink,field_solutiontype,field_solutiondataurl,Key-Words
0,$finance,sector financial servicesdata category financi...,/sites/default/files/img_35.jpg,NaN,NaN,NaN,NaN,NaN,6533,/sites/default/files/img_35.jpg,...,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,Finance,NaN,DataSet,NaN,"{'sector': 'financial servicesdata', 'category..."
1,House_price_prediction_final,house_price_prediction_final,/sites/default/files/img_74.jpg,NaN,NaN,NaN,NaN,NaN,6532,/sites/default/files/img_74.jpg,...,/sites/default/files/img_38.jpg,NaN,NaN,NaN,NaN,House _Price_ Prediction,NaN,MLModel,NaN,[house_price_prediction_final]
2,Diabetes_Patients,sector lifescience healthcaredata category hea...,/sites/default/files/img_80.jpg,NaN,NaN,NaN,NaN,NaN,6531,/sites/default/files/img_80.jpg,...,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,Diabetes Data,NaN,DataSet,NaN,"{'sector': 'lifescience healthcaredata', 'cate..."
3,21nov,sector business servicesdata category adaption...,/sites/default/files/img_85.jpg,NaN,NaN,NaN,NaN,NaN,6530,/sites/default/files/img_85.jpg,...,/sites/default/files/img_79.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN,"{'sector': 'business servicesdata', 'category'..."
4,21nov,sector business servicesdata category adaption...,/sites/default/files/img_58.jpg,NaN,NaN,NaN,NaN,NaN,6529,/sites/default/files/img_58.jpg,...,/sites/default/files/img_97.jpg,NaN,NaN,NaN,NaN,publishedtags,NaN,DataSet,NaN,"{'sector': 'business servicesdata', 'category'..."


In [ ]:
df.columns

Index(['title', 'field_solutiondescription', 'field_solutiondescriptionimage',
       'field_solutiondescriptionvideo', 'field_solutioninsightexchange',
       'field_solutionlink', 'field_solutioninfodescription',
       'field_solutioninfomedia', 'nid', 'field_solutiondescriptionimage_1',
       'field_solutiondescriptionvideo_1', 'field_solutioninsightexchange_1',
       'field_solutioninfomedia_1', 'field_solutioninfodescription2',
       'field_solutionmoreinforefimages', 'field_solutiontags',
       'field_solutiondomain', 'field_solutionindustry',
       'field_solutionoutcomeexchange', 'field_solutionshortdescription',
       'field_solutionaccesslink', 'field_solutiontype',
       'field_solutiondataurl', 'Key-Words'],
      dtype='object')

In [ ]:
# title

In [ ]:
df.to_csv("24.1_november_cleanedData.csv")

##demo

In [ ]:
# https://huggingface.co/spaces/nickmuchi/semantic-search-with-retrieve-and-rerank


,Unnamed: 0,field_solutiondescriptionimage,paragraphs
0,0,/sites/default/files/img_35.jpg,"$finance{'sector': 'financial servicesdata', '..."
1,1,/sites/default/files/img_74.jpg,House_price_prediction_final['house_price_pred...
2,2,/sites/default/files/img_80.jpg,Diabetes_Patients{'sector': 'lifescience healt...
3,3,/sites/default/files/img_85.jpg,"21nov{'sector': 'business servicesdata', 'cate..."
4,4,/sites/default/files/img_58.jpg,"21nov{'sector': 'business servicesdata', 'cate..."


In [84]:
import pandas as pd
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")
df1 = df[['title', 'field_solutiondescriptionimage','Key-Words','nid']]

In [83]:
df1 = df[['title', 'field_solutiondescriptionimage','Key-Words']]
df1.head()

,title,field_solutiondescriptionimage,Key-Words
0,$finance,/sites/default/files/img_35.jpg,"{'sector': 'financial servicesdata', 'category..."
1,House_price_prediction_final,/sites/default/files/img_74.jpg,['house_price_prediction_final']
2,Diabetes_Patients,/sites/default/files/img_80.jpg,"{'sector': 'lifescience healthcaredata', 'cate..."
3,21nov,/sites/default/files/img_85.jpg,"{'sector': 'business servicesdata', 'category'..."
4,21nov,/sites/default/files/img_58.jpg,"{'sector': 'business servicesdata', 'category'..."


In [ ]:
def get_rows(word, df):

    word = word.lower()
    indices = []
    for i, row in df.iterrows():
        if word in row['title'].lower() or word in [j.lower() for j in row['Key-Words']]:
            indices.append(i)
    return df.loc[indices]
    
# print(get_rows('Diabetes', df))

In [ ]:
input='diabetes'
df_searched=get_rows(input, df1)
df_searched.head()



,title,field_solutiondescriptionimage,Key-Words
2,Diabetes_Patients,/sites/default/files/img_80.jpg,"{'sector': 'lifescience healthcaredata', 'cate..."
27,kc_Diabetes_data,/sites/default/files/img_41.jpg,"{'sector': 'lifescience healthcaredata', 'cate..."
164,Pima Indian Diabetes Predictor,/sites/default/files/img_32.jpg,"['pima', 'indian', 'diabetes', 'predictor']"
886,diabetes_model,/sites/default/files/img_7.jpg,['diabetes_model']


In [ ]:
##25 november logic

In [85]:
import pandas as pd
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")
df1 = df[['title', 'field_solutiondescriptionimage','Key-Words','nid']]
df1.head()

,title,field_solutiondescriptionimage,Key-Words,nid
0,$finance,/sites/default/files/img_35.jpg,"{'sector': 'financial servicesdata', 'category...",6533
1,House_price_prediction_final,/sites/default/files/img_74.jpg,['house_price_prediction_final'],6532
2,Diabetes_Patients,/sites/default/files/img_80.jpg,"{'sector': 'lifescience healthcaredata', 'cate...",6531
3,21nov,/sites/default/files/img_85.jpg,"{'sector': 'business servicesdata', 'category'...",6530
4,21nov,/sites/default/files/img_58.jpg,"{'sector': 'business servicesdata', 'category'...",6529


In [88]:
def search_for_Keyword(dataframe, search_string):
    search=search_string.lower().split()
    for i in search:
        if i in list(stopwords.words('english')):
            search.remove(i)
    #print(search)
    lengths = {}
    indices_dict = {}
    for j in range(len(search)):
        indices = []
        for i in range(len(df)):
            if (search[j] in df['title'][i]) or (search[j] in df['Key-Words'][i]):
                indices.append(i)
        indices_dict[search[j]] = indices
        lengths[search[j]] = len(indices)
    
    keyword = [i for i in lengths if lengths[i]==max(lengths.values())]
    
    print("The complete items found are " + str(max(lengths.values())) + " for "+keyword[0]+". The top " +"paths are: \n")
    for i in indices_dict[keyword[0]]:  #top n values to display
        print(df["title"][i])
        print(df["field_solutiondescription"][i])
        print(df["field_solutiondescriptionimage"][i])
        print(df["nid"][i])
        print('\n\n')
        
    """    
    for i in range(len(dataframe)):
        if (keyword in df['title'][i]) or (keyword in df['Key-Words'][i]):
            indices.append(i)
    length1 = len(indices)
    #print(length1)
    #for i in range(5):
    #print(df["field_solutiondescriptionimage"][indices[i]])
    
    if length1 >= 5:
        print("The complete items found are " + str(length1) + ". The top "+"5 "+"paths are: \n")
        for i in range(5):
            print(df["field_solutiondescriptionimage"][indices[i]])
    elif (len(indices) >= 1) and (len(indices) < 5):
        print("The complete items found are " + str(length1) + ". The top "+str(length1)+" paths are: \n")
        for i in range(length1):
              print(df["field_solutiondescriptionimage"][indices[i]])
    else:
         print("There is no content against your search!!, Try again with your search!!!")"""

In [89]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
# df = pd.read_csv("CleanedDataKeyWords.csv")

searchStr = input("Enter search word: ") #financial data #Financial data information is very crucial for the organization
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")

search_for_Keyword(df, searchStr)

Enter search word: powerbi
The complete items found are 6 for powerbi. The top paths are: 

Medical Data Analysis Using PowerBI
medical data analysis using powerbi
/sites/default/files/img_26.jpg
6489



Medical Data Analysis Dashboard
aws medical data analysis using powerbi
/sites/default/files/img_46.jpg
6382



Ad Sales Analytics Power BI
solution provider capgemini ad insertions popular shows major source revenue media houses customer engagement opportunity advertisers however media houses often struggle proper targeting ad optimization content analysis hand advertisers face challenges reconciliation total audience delivery programmatic ad sales ad sales analytics solution addresses concerns ad sellers ad slot purchasers analyzing viewership rating data various third party sources country specific broadcasting data application area advertising firms media houses sales marketing operations inside show analysis gain visibility show genre performing weekday part day buy slot trend ana

In [ ]:
# nid

In [ ]:
#step 1
# !pip install happytransformer
from happytransformer import HappyTextToText, TTSettings

happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

# happy_tt = HappyTextToText("T5", "prithivida/grammar_error_correcter_v1")

args = TTSettings(num_beams=5, min_length=1)

# Add the prefix "grammar: " before each input 
result = happy_tt.generate_text(input(), args=args)

print(result.text) # This sentence has bad grammar.


In [ ]:
#step 2

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

import nltk
from nltk.corpus import wordnet

def grammar_corrector(word):
  synonyms = []
  for syn in wordnet.synsets(word):
    for lemma in syn.lemmas():
      synonyms.append(lemma.name())
  if len(synonyms) > 0:
    return synonyms[0]
  else:
    return word

print(grammar_corrector(input())) # write

In [ ]:

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# Load the required text processing libraries
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

 

In [72]:
##step 3

# Define the sentence to be corrected
sentence = input()

# Tokenize the sentence
tokenized_sent = word_tokenize(sentence)

# Tag each tokenized word with its part of speech
tagged_sent = pos_tag(tokenized_sent)

# Correct the sentence based on the part of speech tags
corrected_sent = []
for word in tagged_sent:
    if word[1] == 'VBG':
        corrected_sent.append('been')
    else:
        corrected_sent.append(word[0])

# Print the corrected sentence
print(' '.join(corrected_sent))

KeyboardInterrupt: ignored

In [ ]:
#grammar connect

#sentence similarity qa - nlp



In [ ]:

df.head()


In [ ]:

# https://lucene.apache.org/
 

In [ ]:
#bag of words

In [ ]:
import pandas as pd
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")
df1 = df[['Key-Words']]

In [ ]:
df1.head()

,Key-Words
0,"{'sector': 'financial servicesdata', 'category..."
1,['house_price_prediction_final']
2,"{'sector': 'lifescience healthcaredata', 'cate..."
3,"{'sector': 'business servicesdata', 'category'..."
4,"{'sector': 'business servicesdata', 'category'..."


In [ ]:
df1.shape

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(df1)):
    print(i)
    title = re.sub('[^a-zA-Z]', ' ', df1['Key-Words'][i])
    title = title.lower()
    title = title.split()
    
    title = [ps.stem(word) for word in title if not word in stopwords.words('english')]
    title = ' '.join(title)
    corpus.append(title)
# --------------------------
# corpus


# -------------------------
from sklearn.feature_extraction.text import CountVectorizer


#Creating the bag of words
bow_article = CountVectorizer().fit(corpus)

count_tokens=bow_article.get_feature_names()

article_vect = bow_article.transform(corpus)
count_tokens
df_count_vect=pd.DataFrame(data=article_vect.toarray(),columns=count_tokens)
df_count_vect

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,aaadata,aai,aapl,abandon,abap,abc,abhinav,abhishek,abil,...,yudhbir,yugandhar,yyyi,zczcxcmz,zebra,zero,zip,zipcod,zone,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3389,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
 #faiss


In [ ]:
 
%%capture
!pip install sentence-transformers
!sudo apt-get install libomp-dev
!pip install faiss

## Encode docs using sentence-transformer

In [ ]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
data = [
    'What is your name?',
    'What is your age?',
]
encoded_data = encoder.encode(data)

In [ ]:
#run this cell for csv custom file

import pandas as pd
df1 = pd.read_csv("/content/cdqa_paragraph.csv")
df1.head()

data = df1['paragraphs'].to_numpy()
encoded_data = encoder.encode(data)

## Add to index

In [ ]:
import faiss
import numpy as np

# IndexFlatIP: Flat inner product (for small datasets)
# IndexIDMap: store document ids in the index as well
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.arange(len(data)))

## Search for question

In [ ]:
def search(query, k=1):
    query_vector = encoder.encode([query])
    top_k = index.search(query_vector, k)
    print(top_k)
    return [
        data[_id] for _id in top_k[1][0]
    ]
    
search("How old are you?")

(array([[106.02962]], dtype=float32), array([[2409]]))


["Exp['nan']"]

## Save and load index

In [ ]:
path = './faiss.index'

# Save index
faiss.write_index(index, path)

## Load index back

In [ ]:
index = faiss.read_index(path)
search("age of yours?")

(array([[112.73239]], dtype=float32), array([[2409]]))


["Exp['nan']"]

In [ ]:
index = faiss.read_index(path)
search("'financial servicesdata")

(array([[139.61205]], dtype=float32), array([[1401]]))


["BankDataStudio['claimsipdata']"]

In [ ]:
#nltk
import nltk
from nltk.tokenize import word_tokenize

import pandas as pd
df = pd.read_csv("/content/cdqa_paragraph.csv")
df.head()




,Unnamed: 0,field_solutiondescriptionimage,paragraphs
0,0,/sites/default/files/img_35.jpg,"$finance{'sector': 'financial servicesdata', '..."
1,1,/sites/default/files/img_74.jpg,House_price_prediction_final['house_price_pred...
2,2,/sites/default/files/img_80.jpg,Diabetes_Patients{'sector': 'lifescience healt...
3,3,/sites/default/files/img_85.jpg,"21nov{'sector': 'business servicesdata', 'cate..."
4,4,/sites/default/files/img_58.jpg,"21nov{'sector': 'business servicesdata', 'cate..."


In [ ]:
  >>> import nltk
  >>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df['paragraphs1'] = df['paragraphs'].apply(word_tokenize)


In [ ]:
df.head()

,Unnamed: 0,field_solutiondescriptionimage,paragraphs,paragraphs1
0,0,/sites/default/files/img_35.jpg,"$finance{'sector': 'financial servicesdata', '...","[$, finance, {, 'sector, ', :, 'financial, ser..."
1,1,/sites/default/files/img_74.jpg,House_price_prediction_final['house_price_pred...,"[House_price_prediction_final, [, 'house_price..."
2,2,/sites/default/files/img_80.jpg,Diabetes_Patients{'sector': 'lifescience healt...,"[Diabetes_Patients, {, 'sector, ', :, 'lifesci..."
3,3,/sites/default/files/img_85.jpg,"21nov{'sector': 'business servicesdata', 'cate...","[21nov, {, 'sector, ', :, 'business, servicesd..."
4,4,/sites/default/files/img_58.jpg,"21nov{'sector': 'business servicesdata', 'cate...","[21nov, {, 'sector, ', :, 'business, servicesd..."


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

df['keyword_tokenized_embeddings'] = df['paragraphs1'].apply(lambda x: tokenizer.encode(x))

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
df.head()

In [ ]:
df.loc[df['paragraphs1'].apply(lambda x: 'business' in x)]

,Unnamed: 0,field_solutiondescriptionimage,paragraphs,paragraphs1,keyword_tokenized_embeddings
